In [9]:
from IPython.display import Audio
from helpers.transform import generate_synthetic_wave, save_wave_to_wav

FREQUENCY = 350
SOUND_DURATION = 0.5
PRE_SILENCE_SECS = 1
POST_SILENCE_SECS = 5

wave, sample_rate = generate_synthetic_wave(
    frequency=FREQUENCY,
    secs_duration=SOUND_DURATION,
    pre_silence_duration=PRE_SILENCE_SECS,
    post_silence_duration=POST_SILENCE_SECS,
)

Audio(wave, rate=sample_rate, autoplay=True)

In [10]:
duration = f"{str(PRE_SILENCE_SECS).zfill(2)}-"
duration += f"{str(SOUND_DURATION).zfill(2)}-"
duration += f"{str(POST_SILENCE_SECS).zfill(2)}"

VOLUMEN = 1
FILENAME = f"trigger-{FREQUENCY}-{duration}.wav"

save_wave_to_wav(wave, sample_rate, FILENAME, VOLUMEN)

In [11]:
from torchaudio import load

audio, audio_sr = load(FILENAME)
Audio(audio, rate=audio_sr, autoplay=True)